# Scrapping

## Scrapping performances (ratings) From footmercato

In [5]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [6]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8'
}

In [7]:
url_barca = 'https://www.footmercato.net/club/8158115007993136624/seasonal-calendar?seasonShortName=2023&currentSeason=2024'
# Send a request to the URL
response_barca = requests.get(url_barca, headers=headers)

In [18]:
perfo_players = []
if response_barca.status_code == 200:

    soup = BeautifulSoup(response_barca.content, 'html.parser')

    month_divs = soup.find_all("a", class_="matchFull__link")
    Links_to_matches = [link["href"] for link in month_divs]
    #matches_id = ["https://www.footmercato.net/"+s.split("-")[0]+"/formation" for s in Links_to_matches]

    for link in Links_to_matches[0:52]:
        response_match = requests.get("https://www.footmercato.net/"+link, headers=headers)
        if response_match.status_code == 200:
            soup_match = BeautifulSoup(response_match.content, 'html.parser')

            match_date = soup_match.find("time", class_="scoreboard__date").text.strip()
            if len(match_date) == 5:
                match_date = match_date+"/2024"
            match_date = datetime.strptime(match_date, "%d/%m/%Y").strftime("%d/%m/%Y")

            # to know barca is away or home team to select the exact div teamformation
            team_d = soup_match.find_all("div", class_="table__row")[8].find("a").text.strip()
            id_notes_joueur = 0 if team_d == "FC Barcelone" else 1

            stats_match = requests.get("https://www.footmercato.net/"+link.split("-")[0]+"/formation", headers=headers)
            if stats_match.status_code == 200:
               soup_stats = BeautifulSoup(stats_match.content, 'html.parser')
               stats_barca = soup_stats.find_all("div", class_="teamFormation")[id_notes_joueur]
               joueurs_barca = stats_barca.find_all("li", class_="teamFormation__player")

               for joueur in joueurs_barca:
                    name_joueur = joueur.find("span", class_="matchTeamPlayer__name").text.strip()

                    notes_joueur_span = joueur.find("span", class_="rating")
                    if notes_joueur_span:
                        notes_joueur = notes_joueur_span.text.strip()
                    else:
                        notes_joueur = None

                    perfo_players.append({
                        'Match_date': match_date,
                        'Player': name_joueur,
                        'Rating': notes_joueur
                    })

In [19]:
df_perfo_players = pd.DataFrame(perfo_players)
df_perfo_players

,Match_date,Player,Rating
0,26/05/2024,M. ter Stegen,7.8
1,26/05/2024,João Cancelo,7.3
2,26/05/2024,Iñigo Martínez,5.7
3,26/05/2024,Pau Cubarsí,6.1
4,26/05/2024,J. Koundé,5.3
...,...,...,...
567,13/08/2023,Oriol Romeu,6
568,13/08/2023,İ. Gündoğan,5.6
569,13/08/2023,Pedri,6.5
570,13/08/2023,Raphinha,4.9


### spliting player name

In [20]:
df_perfo_players['Firstname'] = df_perfo_players['Player'].str.split(' ').str[0]
df_perfo_players['Lastname'] = df_perfo_players['Player'].apply(lambda x: x.split(' ')[0] if len(x.split(' ')) == 1 else " ".join(x.split(' ')[1:]))
del df_perfo_players["Player"]

In [21]:
df_perfo_players.head()

,Match_date,Rating,Firstname,Lastname
0,26/05/2024,7.8,M.,ter Stegen
1,26/05/2024,7.3,João,Cancelo
2,26/05/2024,5.7,Iñigo,Martínez
3,26/05/2024,6.1,Pau,Cubarsí
4,26/05/2024,5.3,J.,Koundé


### Reorginizing columns

In [22]:
new_ordre = ["Match_date", "Firstname", "Lastname", "Rating"]

df_perfo_players = df_perfo_players[new_ordre]

df_perfo_players.head()

,Match_date,Firstname,Lastname,Rating
0,26/05/2024,M.,ter Stegen,7.8
1,26/05/2024,João,Cancelo,7.3
2,26/05/2024,Iñigo,Martínez,5.7
3,26/05/2024,Pau,Cubarsí,6.1
4,26/05/2024,J.,Koundé,5.3


In [23]:
df_perfo_players.to_csv('Performances.csv', index=False)